# Сравнение алгоритмов многокритериальной оптимизации

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm


from MultiobjectiveOptimization.optimizers.moamp import MOAMP
from MultiobjectiveOptimization.optimizers.moea_d import MOEA_D
from MultiobjectiveOptimization.optimizers.nsga2 import NSGA_II
from MultiobjectiveOptimization.optimizers.mopso import MOPSO
from MultiobjectiveOptimization.core.solution import Solution

In [2]:
# Шаг 1: Определение тикеров криптовалют
TICKERS = ["BTC-USD", "ETH-USD", "SOL-USD", "DOGE-USD", "TON11419-USD",
            "SUI20947-USD", "APT21794-USD", "ARB11841-USD", "GRT6719-USD",
        ]

# Шаг 2: Загрузка исторических данных цен
data = yf.download(TICKERS, start="2024-08-01", end="2024-11-12", progress=False)["Adj Close"]

# Шаг 3: Вычисление дневной доходности
returns = data.pct_change().dropna()

# Шаг 4: Вычисление ожидаемой доходности и ковариационной матрицы
expected_returns = returns.mean()
cov_matrix = returns.cov()

In [3]:
# Количество активов
num_assets = len(TICKERS)

# Шаг 5: Определение границ переменных (0, 1) для каждой криптовалюты
variable_bounds = [(0.0, 10.0) for _ in range(num_assets)]


In [4]:
# Шаг 6: Определение функций цели

# Функция прибыли (для максимизации)
def sharpe_ratio_function(weights):
    weights = np.array(weights)
    total_weight = np.sum(weights)
    if total_weight == 0:
        return 0
    weights = weights / total_weight
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    risk_free_rate = 0.01  # Предположим безрисковую ставку 1%
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return sharpe_ratio  # Максимизируем коэффициент Шарпа


# Функция риска (для минимизации)
def var_function(weights):
    weights = np.array(weights)
    total_weight = np.sum(weights)
    if total_weight == 0:
        return np.inf
    weights = weights / total_weight
    portfolio_mean = np.sum(expected_returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    confidence_level = 0.99
    z_score = norm.ppf(1 - confidence_level)
    var = -(portfolio_mean + z_score * portfolio_std)
    return var  # Минимизируем VaR

In [5]:
# Список целевых функций
objectives = [
    {'function': sharpe_ratio_function, 'minimize': False, 'name': 'Прибыль'},  # Прибыль
    {'function': var_function, 'minimize': True, 'name':'Риск'}     # Риск
]

### Используя разные алгоритмы соберём криптовалютные портфели

In [6]:
# MOPSO
mopso = MOPSO(
    population_size=500,
    max_iterations=500,
    variable_bounds=variable_bounds,
    objectives=objectives,
    inertia_weight=0.9,
    cognitive_constant=1.6,
    social_constant=1.2,
    repository_size=60
)

mopso.run()
mopso_front = mopso.get_pareto_front()

mopso.visualize_pareto_front()

Итерация 1/500
Итерация 2/500
Итерация 3/500
Итерация 4/500
Итерация 5/500
Итерация 6/500
Итерация 7/500
Итерация 8/500
Итерация 9/500
Итерация 10/500
Итерация 11/500
Итерация 12/500
Итерация 13/500
Итерация 14/500
Итерация 15/500
Итерация 16/500
Итерация 17/500
Итерация 18/500
Итерация 19/500
Итерация 20/500
Итерация 21/500
Итерация 22/500
Итерация 23/500
Итерация 24/500
Итерация 25/500
Итерация 26/500
Итерация 27/500
Итерация 28/500
Итерация 29/500
Итерация 30/500
Итерация 31/500
Итерация 32/500
Итерация 33/500
Итерация 34/500
Итерация 35/500
Итерация 36/500
Итерация 37/500
Итерация 38/500
Итерация 39/500
Итерация 40/500
Итерация 41/500
Итерация 42/500
Итерация 43/500
Итерация 44/500
Итерация 45/500
Итерация 46/500
Итерация 47/500
Итерация 48/500
Итерация 49/500
Итерация 50/500
Итерация 51/500
Итерация 52/500
Итерация 53/500
Итерация 54/500
Итерация 55/500
Итерация 56/500
Итерация 57/500
Итерация 58/500
Итерация 59/500
Итерация 60/500
Итерация 61/500
Итерация 62/500
Итерация 63/500
И

In [7]:
# MOEA/D

moea_d = MOEA_D(
    population_size=700,
    max_generations=300,
    variable_bounds=variable_bounds,
    objectives=objectives,
    mutation_rate=0.1,
    crossover_rate=0.9,
    nr=2,
    delta=0.9
)

moea_d.run()

moea_d_front = moea_d.get_pareto_front()
moea_d.visualize_pareto_front()

Поколение 1/300
Поколение 2/300
Поколение 3/300
Поколение 4/300
Поколение 5/300
Поколение 6/300
Поколение 7/300
Поколение 8/300
Поколение 9/300
Поколение 10/300
Поколение 11/300
Поколение 12/300
Поколение 13/300
Поколение 14/300
Поколение 15/300
Поколение 16/300
Поколение 17/300
Поколение 18/300
Поколение 19/300
Поколение 20/300
Поколение 21/300
Поколение 22/300
Поколение 23/300
Поколение 24/300
Поколение 25/300
Поколение 26/300
Поколение 27/300
Поколение 28/300
Поколение 29/300
Поколение 30/300
Поколение 31/300
Поколение 32/300
Поколение 33/300
Поколение 34/300
Поколение 35/300
Поколение 36/300
Поколение 37/300
Поколение 38/300
Поколение 39/300
Поколение 40/300
Поколение 41/300
Поколение 42/300
Поколение 43/300
Поколение 44/300
Поколение 45/300
Поколение 46/300
Поколение 47/300
Поколение 48/300
Поколение 49/300
Поколение 50/300
Поколение 51/300
Поколение 52/300
Поколение 53/300
Поколение 54/300
Поколение 55/300
Поколение 56/300
Поколение 57/300
Поколение 58/300
Поколение 59/300
Поколе

In [8]:
# NSGA-II

nsga_2 = NSGA_II(
    population_size=1000,
    max_generations=300,
    variable_bounds=variable_bounds,
    objectives=objectives,
    mutation_rate=0.25
)

nsga_2.run()

nsga_2_front = nsga_2.get_pareto_front()

nsga_2.visualize_pareto_front()

Поколение 1/300
Поколение 2/300
Поколение 3/300
Поколение 4/300
Поколение 5/300
Поколение 6/300
Поколение 7/300
Поколение 8/300
Поколение 9/300
Поколение 10/300
Поколение 11/300
Поколение 12/300
Поколение 13/300
Поколение 14/300
Поколение 15/300
Поколение 16/300
Поколение 17/300
Поколение 18/300
Поколение 19/300
Поколение 20/300
Поколение 21/300
Поколение 22/300
Поколение 23/300
Поколение 24/300
Поколение 25/300
Поколение 26/300
Поколение 27/300
Поколение 28/300
Поколение 29/300
Поколение 30/300
Поколение 31/300
Поколение 32/300
Поколение 33/300
Поколение 34/300
Поколение 35/300
Поколение 36/300
Поколение 37/300
Поколение 38/300
Поколение 39/300
Поколение 40/300
Поколение 41/300
Поколение 42/300
Поколение 43/300
Поколение 44/300
Поколение 45/300
Поколение 46/300
Поколение 47/300
Поколение 48/300
Поколение 49/300
Поколение 50/300
Поколение 51/300
Поколение 52/300
Поколение 53/300
Поколение 54/300
Поколение 55/300
Поколение 56/300
Поколение 57/300
Поколение 58/300
Поколение 59/300
Поколе

In [9]:
# MOAMP
moamp = MOAMP(
    variable_bounds=variable_bounds,
    objectives=objectives,
    population_size=20,
    step_size=0.05,
    tabu_length=30
)

moamp.run(iterations=120)
moamp_front = moamp.get_pareto_front()

moamp.visualize_pareto_front()

100%|██████████| 120/120 [36:35<00:00, 18.30s/it]

Алгоритм MOAMP завершил работу.


# Проанализируем результаты различных алгоритмов

In [10]:
def get_best_and_average_solutions(solutions: list[Solution]) -> list[Solution]:
    """
    Находит лучший по каждой метрике объект и самый средний по всем метрикам объект.

    Параметры:
    ----------
    solutions : List[Solution]
        Список объектов решений.

    Возвращает:
    ----------
    Tuple[Dict[str, Solution], Solution]
        Словарь с лучшими решениями для каждой метрики и самое среднее решение.
    """
    if not solutions:
        return {}, None  # Если список пуст, возвращаем пустые результаты

    # Извлекаем информацию о целевых функциях
    objectives = solutions[0].objectives
    num_objectives = len(objectives)
    objective_names = [obj['name'] for obj in objectives]
    minimize_flags = [obj['minimize'] for obj in objectives]

    # Собираем все значения целевых функций в numpy массив
    objective_values = np.array([sol.objective_values for sol in solutions])  # shape: (num_solutions, num_objectives)

    # Находим лучший объект для каждой метрики
    best_solutions = {}  # key: имя метрики, value: лучшее решение

    for i in range(num_objectives):
        if minimize_flags[i]:
            # Целевая функция минимизируется
            best_index = np.argmin(objective_values[:, i])
        else:
            # Целевая функция максимизируется
            best_index = np.argmax(objective_values[:, i])

        best_solutions[objective_names[i]] = solutions[best_index]

    # Находим самое среднее решение по всем метрикам
    # Нормализуем значения целевых функций между 0 и 1
    normalized_values = np.zeros_like(objective_values)

    for i in range(num_objectives):
        vals = objective_values[:, i]
        min_val = vals.min()
        max_val = vals.max()

        if min_val == max_val:
            # Если все значения одинаковы, устанавливаем 0.5
            normalized_values[:, i] = 0.5
            continue

        if minimize_flags[i]:
            # Для минимизации инвертируем значения (лучшее значение соответствует 1)
            normalized_values[:, i] = (max_val - vals) / (max_val - min_val)
        else:
            # Для максимизации нормализуем прямо
            normalized_values[:, i] = (vals - min_val) / (max_val - min_val)

    # Вычисляем среднее нормализованное значение для каждого решения
    average_normalized_values = normalized_values.mean(axis=1)

    # Находим решение, наиболее близкое к среднему значению
    mean_normalized_value = average_normalized_values.mean()
    difference_from_mean = np.abs(average_normalized_values - mean_normalized_value)
    most_average_index = np.argmin(difference_from_mean)
    most_average_solution = solutions[most_average_index]
    
    result = list(best_solutions.values())
    result.append(most_average_solution)

    return result

def solutions_to_dataframe(solutions: list[Solution], tickers: list[str]) -> pd.DataFrame:
    data = []
    for sol in solutions:
        if len(sol.variables) != len(tickers):
            raise ValueError("Количество variables в Solution не соответствует количеству tickers")
        row = {}
        for ticker, weight in zip(tickers, sol.variables):
            row[ticker] = weight
        for obj, val in zip(sol.objectives, sol.objective_values):
            row[obj['name']] = val
        data.append(row)
    df = pd.DataFrame(data)
    ticker_columns = tickers
    objective_names = [obj['name'] for obj in solutions[0].objectives]
    df = df[ticker_columns + objective_names]
    return df

In [11]:
mopso_data = solutions_to_dataframe(get_best_and_average_solutions(mopso_front), TICKERS)
mopso_data['algorithm'] = 'mopso'

nsga_2_data = solutions_to_dataframe(get_best_and_average_solutions(nsga_2_front), TICKERS)
nsga_2_data['algorithm'] = 'nsga_2'

moea_d_data = solutions_to_dataframe(get_best_and_average_solutions(moea_d_front), TICKERS)
moea_d_data['algorithm'] = 'moea_d'

moamp_data = solutions_to_dataframe(get_best_and_average_solutions(moamp_front), TICKERS)
moamp_data['algorithm'] = 'moamp'

results = pd.concat([mopso_data, nsga_2_data, moea_d_data, moamp_data])

In [13]:
results.sort_values(by='Прибыль', ascending=False)

,BTC-USD,ETH-USD,SOL-USD,DOGE-USD,TON11419-USD,SUI20947-USD,APT21794-USD,ARB11841-USD,GRT6719-USD,Прибыль,Риск,algorithm
0,-0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,1.0257,-0.0000,0.104540,0.156990,mopso
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.104540,0.156990,nsga_2
2,0.0010,0.0000,0.0000,0.0474,0.0088,0.0019,0.0076,1.0485,0.0067,0.100539,0.150261,mopso
2,0.0000,0.0000,0.0000,0.3278,0.0000,0.0000,0.0000,0.6722,0.0000,0.096548,0.127969,nsga_2
0,0.0086,0.0018,0.0375,0.0460,0.0238,0.0375,0.0000,0.8214,0.0232,0.083154,0.138940,moamp
0,0.0000,0.0000,0.2279,0.0000,0.0064,0.0000,0.0000,0.7504,0.0153,0.068041,0.128438,moea_d
2,0.0332,0.0567,0.1329,0.3427,0.0712,0.0090,0.0856,0.1723,0.0964,-0.038721,0.090560,moamp
1,0.1364,0.0717,0.2060,0.1858,0.1277,0.0207,0.0797,0.0119,0.1601,-0.141339,0.080881,moamp
1,0.0000,0.0000,1.1407,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.220998,0.065754,mopso
2,0.0000,0.0000,0.9674,0.0000,0.0005,0.0000,0.0000,0.0000,0.0321,-0.227898,0.065325,moea_d
